In [104]:
# Import potrebných knihoven
import pandas as pd
import numpy as np
from scipy.stats import chi2_contingency

# Načtení datasetu
df_accidents = pd.read_pickle("accidents.pkl.gz")

# Test hypotézy (xhejni00)

## 1. příklad

Zadaní (hypotéza 1): Na silnicích první třídy byly nehody s následky na zdraví se stejnou pravděpodobností
jako na dálnicích.

Hypotéza 1 je zároveň hypotézou nula pro náš chí kvadrát test. H0: Na silnicích I. třídy byly dané nehody se stejnou pravděpodobností jako na dálnicích
Hypotéza H1: Na silnicích I. třídy nejsou dané nehody se stejnou pravděpodobností jako na dálnících.

Daným nehodám rozumějme nehody s následky na zdraví.

Nejprve vyfiltrujeme pouze nehody, které se staly na dálnici nebo silnici první třídy

In [105]:
df_accidents = df_accidents[df_accidents['p36'].isin([0, 1])]

Dále vytvoříme dva binární sloupce "highway" a "injury". Hodnoty ve sloupci "highway" nabývají hodnoty True pokud se jedná o dálnici, False v opačném případě. Hodnoty v druhém slpupci nabývají hodnoty True pokud došlo k nehodě s následky na zdraví, False v opačném případě

In [106]:
df_accidents['highway'] = df_accidents['p36'] == 0 
df_accidents['injury'] = df_accidents['p9'] == 1

Dalším krokem je vytvoření kontigenční tabulky pro dané sloupce.

In [107]:
ct = pd.crosstab(df_accidents['highway'], df_accidents['injury'])
ct

injury,False,True
highway,,
False,14773,7059
True,6674,1247


Tuto kontingenční tabulku využijeme pro chí kvadrát test, který je spočítán funkcí chi2_contingency. P-value získanou tímto testem použijeme pro prokázání nebo zamítnutí nulové hypotézy.
Zadání požaduje hladinu spolehlivosti 95 %, tedy alfa je 0.05. Proti této hodně dále postavíme získanou p-hodnotu.

In [108]:
result = chi2_contingency(ct)
print(f'P-hodnota: {result.pvalue}')

if result.pvalue < 0.05:
    print("Zamítáme nulovou hypotézu, tedy jsme nepotvrdili stejnou míru zranění na dálnici a na silnicích první třídy.")
else:
    print("Nezamitáme nulovou hypotézu, a tedy je pravděpodobnost daných nehod s následky na zdraví na obou typech silnic stejná.")

P-hodnota: 1.0075002730159729e-174
Zamítáme nulovou hypotézu, tedy jsme nepotvrdili stejnou míru zranění na dálnici a na silnicích první třídy.


Vzhledem k výsledku p-value, která je velice malá (asi 1.0e-174) zamítáme nulovou hypotézu, tedy říkáme že s 95 % jistotou nejsou na obou typech silnic nehody s následky na zdraví se stejnou pravděpodobností.

V neposlední řadě zodpovíme otázku, zdali nehody na silnicích 1. třídy vedly méně nebo více často k nehodě s následky na zdraví. K tomu využijeme výstup expected, který vrací již dříve použitá funkce pro výpočet testu. 

In [109]:
print(result.expected_freq)

[[15737.26696468  6094.73303532]
 [ 5709.73303532  2211.26696468]]


Tato tabulka nám říká očekávané hodnoty v kontingenční tabulce, aby nedošlo k zamítnutí nulové hypotézy. V našem i tomto případě lze říci, že na silnicích I. třídy dochází k nehodám s následky na zdraví častěji než na dálnicích.